In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [3]:
toner_url = []

In [4]:
for i in range (1,68):
    url = f'https://www.stylevana.com/en_EU/skincare/face-care/toner-mist.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    toner_list = soup.find_all('div', class_='product-item-info')
    for item in toner_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            toner_url.append(link['href'])
print(len(toner_url))

790


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [5]:
toners = []

In [6]:
for link in toner_url[0:790]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name = soup.find('h1', class_='product-name-h1').text
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        price = (soup.find('span', class_='price').text.strip())
        best_seller = soup.find(class_="product-badge-icon").get('title')
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        details = 'no info'
        ingredients = 'no info'
        price = 'no info'
        best_seller = 'no'
        
    toners_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients,
    'price': price,
    'best-seller': best_seller}
    
    toners.append(toners_info)

In [7]:
print(len(toners))

790


## 3. PASARLO A DATAFRAME

In [8]:
toners_df = pd.DataFrame(toners)
toners_df

,name,details,ingredients,price,best-seller
0,Isntree - Green Tea Fresh Toner,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Camellia Sinensis Leaf Extract, Water, Ginkgo ...",1.09€,Cruelty Free
1,COSRX - Centella Water Alcohol-Free Toner 150ml,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Water, Centella Asiatica Leaf Water, Butylene ...",12.39€,Best Sellers
2,Isntree - Hyaluronic Acid Toner Plus,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...",[],9.99€,Cruelty Free
3,SOME BY MI - AHA-BHA-PHA 30 Days Miracle Toner...,"Usage: Day, Night\n\n\nProduct Type: Toner & M...",[],14.09€,Best Sellers
4,Haruharu WONDER - Black Rice Hyaluronic Toner ...,"Usage: Day, Night\n\n\nProduct Type: Toner & M...","After washing the face, wet a cotton pad with ...",11.09€,Best Sellers
...,...,...,...,...,...
785,It's Skin - Cera Routine Essential Toner - 200ml,no info,no info,no info,no
786,The Saem - Eco Energy Fresh Toner - 150ml,no info,no info,no info,no
787,innisfree - Olive Real Oil Mist EX - 80ml,"Usage: Day, Night\n\n\nProduct Type: Toner & M...","Water, Butylene Glycol, Olea Europaea (Olive) ...",5.99€,Clearance
788,A.H.C - Hydra G6 Essence Toner - 130ml,no info,no info,no info,no


In [9]:
toners = toners_df.to_csv('../data_/toners.csv')